In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import rich
import requests
import json

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

In [3]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

Calling a function and sending the result back to Chat API and Responses API

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

In [4]:
def get_weather_function_chat():
    return {
        "type": "function",
        "function": { # This property is removed from responses API
            "name": "get_weather",
            "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location"
                    }
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

In [5]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
]
tools = [get_weather_function_chat()]
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools = tools
)

rich.print(response.choices[0])
print("Finish Reason = ", response.choices[0].finish_reason)
rich.print(response.choices[0].message.tool_calls)


Choice(
    finish_reason='tool_calls',
    index=0,
    logprobs=None,
    message=ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_P8uOzPkQ84LoooK3DEOmcgCx',
                function=Function(arguments='{"latitude":24.8607,"longitude":67.0011}', name='get_weather'),
                type='function'
            )
        ]
    )
)

Finish Reason =  tool_calls


[
    ChatCompletionMessageToolCall(
        id='call_P8uOzPkQ84LoooK3DEOmcgCx',
        function=Function(arguments='{"latitude":24.8607,"longitude":67.0011}', name='get_weather'),
        type='function'
    )
]

In [6]:
if response.choices[0].finish_reason == "tool_calls": # Check if finish_reason is tool_calls
    tool_call = response.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = { # Properties of this object will be different in responses API
        "role": "tool",
        "content": json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
        "tool_call_id": tool_call.id
    }
    messages.append(response.choices[0].message) # Important: we will append the previous message (response.choices[0].message)
    messages.append(new_message)
    response2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("Model Response2 = ",response2.choices[0].message.content)
    print("Finish Reason = ",response2.choices[0].finish_reason)

get_weather function called to get weather for latitude = 24.8607, longitude = 67.0011
And result is  = 30.1
Model Response2 =  The current weather in Karachi, Pakistan, is approximately 30.1°C.
Finish Reason =  stop


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses


Note: Check the difference between json objects for function

In [7]:
def get_weather_function_response():
    return {
        "type": "function",  # There is no function property in the response API
        "name": "get_weather",
        "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "Latitude of the location"
                },
                "longitude": {
                    "type": "number",
                    "description": "Longitude of the location"
                }
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }

### Using old way of sending history messages in every call

In [8]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
     {"role": "user", "content": "Berline"}
]
tools = [get_weather_function_response()]

response = openai.responses.create(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

Status =  completed



[
    ResponseFunctionToolCall(
        id='fc_67da97283c508192a2b1c847af8580210d30e3bc2f652424',
        arguments='{"latitude":52.520008,"longitude":13.404954}',
        call_id='call_QarU5K9lxymSXwfpqywf28Eg',
        name='get_weather',
        type='function_call',
        status='completed'
    )
]

Note: `new_message` has different property names.

```
new_message = {
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(weather),
}
```

In [9]:
if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    arguments = json.loads(tool_call.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        # "output": str(weather)
        # Because of json object in output Responses API sometimes does not generate expected output
        "output":  json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
    }
    messages.append(response.output[0]) # Important: we will append the tool call (response.output[0])
    messages.append(new_message)
    response2 = openai.responses.create(model=MODEL, input=messages,tools = tools)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)

get_weather function called to get weather for latitude = 52.520008, longitude = 13.404954
And result is  = 5.3
Model Response2 =  The current temperature in Berlin is 5.3°C. If you need more details like humidity or precipitation, let me know!
Status =  completed


### Using new way of conversation state by sending perivous reponse id

In [ ]:
# This section is same as above

messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
     {"role": "user", "content": "Berline"}
]
tools = [get_weather_function_response()]

response = openai.responses.create(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

Status =  completed



[
    ResponseFunctionToolCall(
        id='fc_67da977e6d288192b4b6df836f7aabe70ef8d4ed3645e53c',
        arguments='{"latitude":52.520008,"longitude":13.404954}',
        call_id='call_lO75Czenu2RC1xwkOkuAcGPA',
        name='get_weather',
        type='function_call',
        status='completed'
    )
]

In [ ]:
# The only difference in this section is how messages are sent.

if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    arguments = json.loads(tool_call.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        # "output": str(weather)
        # Because of json object in output Responses API sometimes does not generate expected output
        "output":  json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
    }
    # messages.append(response.output[0]) # Not needed now
    messages.append(new_message)
    response2 = openai.responses.create(model=MODEL, input=messages,tools = tools, previous_response_id=response.id)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)

get_weather function called to get weather for latitude = 52.520008, longitude = 13.404954
And result is  = 5.3
Model Response2 =  The current weather in Berlin is 5.3°C. If you need more specific details like wind speed or conditions, let me know!
Status =  completed
